# Create Train-Val-Test
 Joins monthly parquets files into one single parquet used for model train and evaluation
 Manually complete file names is needed

In [1]:
!pip install pyathena

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 9.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [3]:
# Set to zero if train set and to one for test
archivo_test = 1

if archivo_test == 0: 
    s3_url = f"viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Train"
elif archivo_test == 1:
    s3_url = f"viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Test"
elif archivo_test == 2:
    s3_url = f"viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Test"

## Create Train & Validacion
Load parquet for each month

In [4]:
if archivo_test == 0: 
    s3_url = f"viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Train"
    
    df1 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202301.parquet")
    df2 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202302.parquet")
    df3 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202303.parquet")
    df4 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202304.parquet")
    df5 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202305.parquet")
    df6 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202306.parquet")
    df7 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202307.parquet")
    df8 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202308.parquet")
    #df12 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202212.parquet")
    #df11 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202211.parquet")
    

In [5]:
## join all dataframes
if archivo_test == 0:
    dfs = [df1, df2, df3, df4, df5, df6, df7, df8] #df11, df12

    df = pd.concat(dfs, ignore_index=True)

In [6]:
# Corrijo una normalizacion que me falto
# DB Setting
from pyathena import connect
import numpy as np

bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'
origin_name = 'AwsDataCatalog'
database_name = 'analytics'
tablaSource = 'source_fraud_fechas_label'

#Connection
conn = connect(s3_staging_dir=f'{bucket_name}queryresults/',
               region_name='us-east-1')

df_perc = pd.read_sql('''select 'branch_working_days', ptile, min(branch_working_days), max(branch_working_days)
            from (select NTILE(100) OVER (order by cast(branch_working_days as integer)) as ptile, 
                    cast(branch_working_days as integer) as branch_working_days
                from analytics.abt_fraud
                where date_receiver between cast('2023-01-01' as timestamp) and cast('2023-08-01' as timestamp))
            where ptile = 1 or ptile = 99
            group by ptile;''', conn)
df_perc = df_perc.rename(columns={"_col0": "Variable", "_col2": "Min", "_col3": "Max", "_col4": "Avg"})

def scale_percentils(data, Pmin, Pmax, calculo):
    """
    Define scale function

    Parameters:
    - data: Column to be scale.
    - calculo: Whether data is scale using min and max value or predefined value
    - Pmin: Value for inferior limit when calculo=0 or percentil for inferior limit when calculo=1 
    - Pmax: Value for superior limit when calculo=0 or percentil for superior limit when calculo=1 
    - calculo = 1 data is scaled using specified percentiles, Pmin & Pmax, as minimun and maximun
    - calculo = 0 data is scaled using values given by Pmin and Pmax values

    Returns:
    - dataN: scaled data
    """
    m = data.min()
    M = data.max()
    if calculo==1:
        Qmin = Pmin
        Qmax = Pmax
        q0 = data.quantile(Qmin)
        q1 = data.quantile(Qmax)
    elif calculo==0:
            q0=Pmin
            q1=Pmax
    data_std = (data - q0) / (q1 - q0)
    dataN =  data_std.to_numpy()
    #Set limits for outliers
    dataN = np.clip(dataN,0,1)
    return dataN


Mx = df_perc.Max[(df_perc['Variable']=='branch_working_days') & (df_perc['ptile']==99)].item()
mn = df_perc.Min[(df_perc['Variable']=='branch_working_days') & (df_perc['ptile']==1)].item()
Mx = int(Mx)
mn = int(mn)

#if archivo_test == 0:
#    df['01_branch_working_days'] = scale_percentils(df['branch_working_days'].astype(int), mn, Mx, 0)


/tmp/ipykernel_1642/2873324813.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_perc = pd.read_sql('''select 'branch_working_days', ptile, min(branch_working_days), max(branch_working_days)


In [7]:
if archivo_test == 0:
    from sklearn.model_selection import train_test_split

    X_train, X_val, y_train, y_val = train_test_split(
        df.loc[:, df.columns != 'target_fraudes'], df['target_fraudes'], test_size=0.3, random_state=42
    )

In [8]:
if archivo_test == 0:
    #Feature names and traget column
    feature_names = df.loc[:, df.columns != 'target_fraudes'].columns

    trainX = pd.DataFrame(X_train, columns=feature_names)
    trainX["target"] = y_train

    valX = pd.DataFrame(X_val, columns=feature_names)
    valX["target"] = y_val

In [9]:
# save to parquet
if archivo_test == 0:
    s3_url = f"s3://viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Data4Model/TargetTodos/Train/Train.parquet"
    trainX.to_parquet(path=s3_url, engine='pyarrow', compression='gzip')

    s3_url = f"s3://viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Data4Model/TargetTodos/Validation/Validation.parquet"
    valX.to_parquet(path=s3_url, engine='pyarrow', compression='gzip')
    print('finish saving parquet of train & val')

## Create test for model

In [10]:
if archivo_test == 1: 
    s3_url = f"viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Test"
    
    #dfT1 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202308.parquet")
    dfT2 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202309.parquet")
    dfT3 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202310.parquet")
    dfT4 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202311.parquet")
    dfT5 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202312.parquet")
    

In [11]:
## join months
if archivo_test == 1:
    dfTs = [ dfT2, dfT3, dfT4, dfT5] #dfT1,

    dfT = pd.concat(dfTs, ignore_index=True)
    dfT = dfT.rename(columns={"target_fraudes": "target"})
    
    #dfT['receiver_transaction_count'] = dfT['receiver_transaction_count'].astype(str)
    #dfT['sender_sending_days'] = dfT['sender_sending_days'].astype(str)
    #dfT['01_branch_working_days'] = scale_percentils(dfT['branch_working_days'].astype(int), mn, Mx, 0)
    

    s3_url = f"s3://viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Data4Model/TargetTodos/Test/Test.parquet"
    dfT.to_parquet(path=s3_url, engine='pyarrow', compression='gzip')
    print('finish saving parquet test')

finish saving parquet test


## Create 2024 data for model

In [12]:
if archivo_test == 2: 
    s3_url = f"viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Test"
    
    #dfT1 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202308.parquet")
    dfT2401 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202401.parquet")
    dfT2402 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202402.parquet")
    dfT2403 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202403.parquet")

In [13]:
## join months
if archivo_test == 2:
    dfTs = [ dfT2401, dfT2402, dfT2403] 

    dfT = pd.concat(dfTs, ignore_index=True)
    dfT = dfT.rename(columns={"target_fraudes": "target"})
    print(dfT['target'].value_counts())

    s3_url = f"s3://viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Data4Model/TargetTodos/Data2024/Data2024.parquet"
    dfT.to_parquet(path=s3_url, engine='pyarrow', compression='gzip')
    print('finish saving parquet 2024')

In [14]:
import seaborn as sns
fig, axs = plt.subplots(1,2,figsize = (12, 4))
sns.histplot(data=dfT, x="branch_working_days", ax = axs[0])
sns.histplot(data=dfT, x="01_branch_working_days", ax = axs[1])

NameError: name 'plt' is not defined